# kNN - Experimento variando K y tamaño de imagenes


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [ ]:
!cd .. && ./build.sh

## Explorando los datos

In [22]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df = pd.read_csv("../data/train.csv")
df.info()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [23]:
df.sample(frac=1)

limit = int(0.8 * df.shape[0])
#df = df[:5000]
print(limit)

33600


In [24]:
df_train = df[:limit]
df_val = df[limit:]


X_t = df_train[df_train.columns[1:]].values
y_t = df_train["label"].values.reshape(-1, 1)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [25]:
class KNNnPCAClassifier:
    def __init__(self, k, alpha):
        self.knn_classifier = metnum.KNNClassifier(k, "distance_pow")
        self.pca = metnum.PCA(alpha)

    def fit(self, X_train, y_train):
        self.pca.fit(X_train)
        self.knn_classifier.fit(self.pca.transform(X_train), y_train)
    
    def predict(self, X):
        return self.knn_classifier.predict(self.pca.transform(X))

### Experimento

In [26]:
import metnum
import pandas as pd
from sklearn import metrics

clf1 = KNNnPCAClassifier(7,38)
clf2 = metnum.KNNClassifier(4, "distance_pow")

clf1.fit(X_t, y_t)
clf2.fit(X_t, y_t)

scores = []

for i in range(10):
    df_digit = df_val[df_val["label"] == i]
    X_val = df_digit[df_digit.columns[1:]].values
    y_val = df_digit["label"].values.reshape(-1, 1)
    y_pred_knnpca = clf1.predict(X_val)
    y_pred_knn = clf2.predict(X_val)
    
    item = {
        "kappa" : metrics.cohen_kappa_score(y_pred_knnpca, y_pred_knn),
        "accuracy_knn_pca" : metrics.accuracy_score(y_pred_knnpca, y_val),
        "accuracy_knn" : metrics.accuracy_score(y_pred_knn, y_val),
        }
    scores.append(item)
    
scores

[{'kappa': 0.6649955765260985,
  'accuracy_knn_pca': 0.9917840375586855,
  'accuracy_knn': 0.994131455399061},
 {'kappa': 0.6660808435852372,
  'accuracy_knn_pca': 0.9957894736842106,
  'accuracy_knn': 0.9978947368421053},
 {'kappa': 0.7800823661549982,
  'accuracy_knn_pca': 0.9725343320848939,
  'accuracy_knn': 0.9637952559300874},
 {'kappa': 0.6179534250531136,
  'accuracy_knn_pca': 0.9659863945578231,
  'accuracy_knn': 0.9705215419501134},
 {'kappa': 0.6454730911210413,
  'accuracy_knn_pca': 0.967940813810111,
  'accuracy_knn': 0.967940813810111},
 {'kappa': 0.6994528190547717,
  'accuracy_knn_pca': 0.965194109772423,
  'accuracy_knn': 0.9665327978580991},
 {'kappa': 0.6909344490934448,
  'accuracy_knn_pca': 0.9939831528279182,
  'accuracy_knn': 0.9903730445246691},
 {'kappa': 0.5658863971190047,
  'accuracy_knn_pca': 0.9807692307692307,
  'accuracy_knn': 0.9796380090497737},
 {'kappa': 0.6402474864655839,
  'accuracy_knn_pca': 0.9600997506234414,
  'accuracy_knn': 0.936408977556109

In [27]:
print("Creating DataFrame...")
df_r = pd.DataFrame(scores)
print("Finished!")

Creating DataFrame...
Finished!


In [28]:
df_r

,accuracy_knn,accuracy_knn_pca,kappa
0,0.994131,0.991784,0.664996
1,0.997895,0.995789,0.666081
2,0.963795,0.972534,0.780082
3,0.970522,0.965986,0.617953
4,0.967941,0.967941,0.645473
5,0.966533,0.965194,0.699453
6,0.990373,0.993983,0.690934
7,0.979638,0.980769,0.565886
8,0.936409,0.960100,0.640247
9,0.955952,0.959524,0.740036


In [29]:
df_r.to_csv("data/kappa.csv", index=True)